# Build a Convolutional Neural Network using Estimators

* Based on this tutorial: https://www.tensorflow.org/tutorials/estimators/cnn

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

CS231n Convolutional Neural Networks for Visual Recognition: https://cs231n.github.io/convolutional-networks/

## Building the CNN MNIST Classifier

Let's build a model to classify the images in the MNIST dataset using the following CNN architecture:

1. **Convolutional Layer**: Applies 32 5x5 filters (extracting 5x5-pixel subregions), with ReLU activation function
2. **Pooling Layer**: Performs max pooling with a 2x2 filter and stride of 2 (which specifies that pooled regions do not overlap)
3. **Convolutional Layer**: Applies 64 5x5 filters, with ReLU activation function
4. **Pooling Layer**: Again, performs max pooling with a 2x2 filter and stride of 2
5. **Dense Layer**: 1,024 neurons, with dropout regularization rate of 0.4 (probability of 0.4 that any given element will be dropped during training)
6. **Dense Layer** (Logits Layer): 10 neurons, one for each digit target class (0–9).

In [4]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [5]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

11493376/11490434 [==============================] - 1s 0us/step


In [6]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

I0806 17:28:29.015386 4729447872 estimator.py:1790] Using default config.
I0806 17:28:29.020840 4729447872 estimator.py:209] Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb364eb908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [8]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])

W0806 17:31:07.749806 4729447872 deprecation.py:323] From /Users/cschmidl/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0806 17:31:07.766159 4729447872 deprecation.py:323] From /Users/cschmidl/anaconda3/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0806 17:31:07.768745 4729447872 deprecation.py:323] From /Users/cschmidl/anaconda3/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/inputs/queues/

I0806 17:31:09.571979 4729447872 basic_session_run_hooks.py:262] loss = 2.2947886, step = 1
I0806 17:31:09.573289 4729447872 basic_session_run_hooks.py:606] Saving checkpoints for 1 into /tmp/mnist_convnet_model/model.ckpt.
I0806 17:31:09.737036 4729447872 estimator.py:368] Loss for final step: 2.2947886.


In [11]:
mnist_classifier.train(input_fn=train_input_fn, steps=2000)

I0806 17:45:41.789334 4729447872 estimator.py:1145] Calling model_fn.
I0806 17:45:41.995869 4729447872 estimator.py:1147] Done calling model_fn.
I0806 17:45:41.998392 4729447872 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0806 17:45:42.065080 4729447872 monitored_session.py:240] Graph was finalized.
I0806 17:45:42.069709 4729447872 saver.py:1280] Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1001
I0806 17:45:42.112931 4729447872 session_manager.py:500] Running local_init_op.
I0806 17:45:42.118672 4729447872 session_manager.py:502] Done running local_init_op.
I0806 17:45:42.276324 4729447872 basic_session_run_hooks.py:606] Saving checkpoints for 1001 into /tmp/mnist_convnet_model/model.ckpt.
I0806 17:45:42.497745 4729447872 basic_session_run_hooks.py:262] loss = 2.0831218, step = 1002
I0806 17:45:48.436609 4729447872 basic_session_run_hooks.py:692] global_step/sec: 16.8376
I0806 17:45:48.437654 4729447872 basic_session_run_hooks.py:260] loss = 1.9834975,

In [12]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

I0806 17:54:39.250916 4729447872 estimator.py:1145] Calling model_fn.
I0806 17:54:39.341981 4729447872 estimator.py:1147] Done calling model_fn.
I0806 17:54:39.361567 4729447872 evaluation.py:255] Starting evaluation at 2019-08-06T17:54:39Z
I0806 17:54:39.428659 4729447872 monitored_session.py:240] Graph was finalized.
I0806 17:54:39.430938 4729447872 saver.py:1280] Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-3001
I0806 17:54:39.475301 4729447872 session_manager.py:500] Running local_init_op.
I0806 17:54:39.483315 4729447872 session_manager.py:502] Done running local_init_op.
I0806 17:54:41.065331 4729447872 evaluation.py:275] Finished evaluation at 2019-08-06-17:54:41
I0806 17:54:41.066061 4729447872 estimator.py:2039] Saving dict for global step 3001: accuracy = 0.8998, global_step = 3001, loss = 0.3958079
I0806 17:54:41.068660 4729447872 estimator.py:2099] Saving 'checkpoint_path' summary for global step 3001: /tmp/mnist_convnet_model/model.ckpt-3001


{'accuracy': 0.8998, 'loss': 0.3958079, 'global_step': 3001}
